##200412_가우시안,자기조직화지도,LVQ,심층학습.ipynb

### 1. 가우시안 혼합 모델(Gaussian Mixture Model)

(1) Gaussian 분포가 K개 혼합된 cluster 알고리즘<br>
(2) Mixture Model: 전체 분포에서 하위 분포가 존재한다는 모델(데이터가 모수를 갖는 여러 개의 분포로부터 생성되었다고 가정)<br>
(3) 가우시안 확률 분포의 한계: 데이터들의 평균을 중심으로 하나의 그룹으로 뭉쳐있는 unimodel한 형태만 표현 가능 -> 한계점 완화하기 위해 GMM 모델 사용.<br>
(4) 사용 분야: 데이터마이닝, 패턴 인식, 머신 러닝, 통계분석 등에 광범위하게 사용 <br><br>

(5) EM 알고리즘을 통해 모델의 파라미터 구함.
- 주어진 데이터가 어디 가우시안에서 생선된 데이터인지?<br>
(KNOW: K개의 가우시안에서 생성된 데이터 + 각 가우시안이 처음 선택될 확률과 가우시안의 parameter) : M 단계
- 가우시안이 선택될 확률과 가우시안들의 parameter 추정<br>
(KNOW: K개의 가우시안에서 생성된 데이터 + 각 데이터가 어떤 가우시안에서 생성되었는지) : E 단계
- 위의 2과정을 계속해서 반복하고, parameter가 더 이상 변하지 않을 때 반복을 stop하면, 우리가 원하는 가우시안 parameter, 선택될 확률, 그리고 데이터가 어디 가우시안에 속했는지를 알 수 있다.

In [0]:
import matplotlib.pyplot as plt
from sklearn import cluster, datasets, mixture
import numpy as np
from scipy.stats import multivariate_normal
from sklearn.datasets import make_spd_matrix
plt.rcParams["axes.grid"] = False

# define the number of samples to be drawn
n_samples = 100

In [0]:
# define the mean points for each of the systhetic cluster centers
t_means = [[8.4, 8.2], [1.4, 1.6], [2.4, 5.4], [6.4, 2.4]]

# for each cluster center, create a Positive semidefinite convariance matrix
t_covs = []
for s in range(len(t_means)):
  t_covs.append(make_spd_matrix(2))

X = []
for mean, cov in zip(t_means,t_covs):
  x = np.random.multivariate_normal(mean, cov, n_samples)
  X += list(x)
  
X = np.array(X)
np.random.shuffle(X)
print("Dataset shape:", X.shape)

Dataset shape: (400, 2)


In [0]:
# Create a grid for visualization purposes 
x = np.linspace(np.min(X[...,0])-1,np.max(X[...,0])+1,100)
y = np.linspace(np.min(X[...,1])-1,np.max(X[...,1])+1,80)
X_,Y_ = np.meshgrid(x,y)
pos = np.array([X_.flatten(),Y_.flatten()]).T
print(pos.shape)
print(np.max(pos[...,1]))

(8000, 2)
11.334920050884445


In [0]:
# define the number of clusters to be learned
k = 4

# create and initialize the cluster centers and the weight paramters
weights = np.ones((k)) / k
means = np.random.choice(X.flatten(), (k,X.shape[1]))
print(means)
print(weights)

[[8.34239963 5.62003589]
 [0.53090515 2.1129898 ]
 [9.42645314 7.34745668]
 [0.77751233 7.80568938]]
[0.25 0.25 0.25 0.25]


In [0]:
# create and initialize a Positive semidefinite convariance matrix 
cov = []
for i in range(k):
  cov.append(make_spd_matrix(X.shape[1]))
cov = np.array(cov)
print(cov.shape)

(4, 2, 2)


In [0]:
colors = ['tab:blue', 'tab:orange', 'tab:green', 'magenta', 'yellow', 'red', 'brown', 'grey']
eps=1e-8

# run GMM for 40 steps
for step in range(40):

  # visualize the learned clusters
  if step % 1 == 0:
    plt.figure(figsize=(12,int(8)))
    plt.title("Iteration {}".format(step))
    axes = plt.gca()
    
    likelihood = []
    for j in range(k):
      likelihood.append(multivariate_normal.pdf(x=pos, mean=means[j], cov=cov[j]))
    likelihood = np.array(likelihood)
    predictions = np.argmax(likelihood, axis=0)
    
    for c in range(k):
      pred_ids = np.where(predictions == c)
      plt.scatter(pos[pred_ids[0],0], pos[pred_ids[0],1], color=colors[c], alpha=0.2, edgecolors='none', marker='s')
    
    plt.scatter(X[...,0], X[...,1], facecolors='none', edgecolors='grey')
    
    for j in range(k):
      plt.scatter(means[j][0], means[j][1], color=colors[j])

    #plt.savefig("img_{0:02d}".format(step), bbox_inches='tight')
    plt.show()

  likelihood = []
  # Expectation step
  for j in range(k):
    likelihood.append(multivariate_normal.pdf(x=X, mean=means[j], cov=cov[j]))
  likelihood = np.array(likelihood)
  assert likelihood.shape == (k, len(X))
    
  b = []
  # Maximization step 
  for j in range(k):
    # use the current values for the parameters to evaluate the posterior
    # probabilities of the data to have been generanted by each gaussian
    b.append((likelihood[j] * weights[j]) / (np.sum([likelihood[i] * weights[i] for i in range(k)], axis=0)+eps))

    # updage mean and variance
    means[j] = np.sum(b[j].reshape(len(X),1) * X, axis=0) / (np.sum(b[j]+eps))
    cov[j] = np.dot((b[j].reshape(len(X),1) * (X - means[j])).T, (X - means[j])) / (np.sum(b[j])+eps)

    # update the weights
    weights[j] = np.mean(b[j])
    
    assert cov.shape == (k, X.shape[1], X.shape[1])
    assert means.shape == (k, X.shape[1])

Output hidden; open in https://colab.research.google.com to view.

* Reference <br>
1. http://blog.naver.com/PostView.nhn?blogId=kmkim1222&logNo=10187825620 <br>
2. https://3months.tistory.com/154 <br>
3. https://colab.research.google.com/drive/1Eb-G95_dd3XJ-0hm2qDqdtqMugLkSYE8 <br>

###2. 자기조직화지도(SOM, Self-Organizing Map)

(1) 대뇌피질의 시각피질을 모델화한 인공신경망 <br>
(2) 고차원 데이터를 사람이 볼 수 있는 2차원, 3차원 격자에 대응하도록 인경신공망과 유사한 방식의 학습을 통해 군지을 도출해내는 기법 -> 고차원의 데이터 원공간에서 유사한 개체들은 저차원에 인접한 격자들과 연결. <br>
(3) Winning node: 임의의 n차원 입력벡터가 들어왔을 대 가장 가까운 격자벡터 -> 군집화 <br>
(4) 같은 격자에 할당된 입력벡터라 하더라도 winning node와 거리가 다름. -> 멀고 가까움 표시하면, 고차원 공간의 데이터를 차원 축소 가능. <br>
(5) 사용 분야: 차원축소(dimensionality reduction), 군집화(clustering)
<br><Br>

![대체 텍스트](https://i.imgur.com/ZsAdHxT.png)
![대체 텍스트](https://i.imgur.com/EE8NF6J.png)





### - SOM 학습방법

(1) 검정색: 원데이터(고차원), 연두색: 격자벡터(저차원) <br>
(2) 격자벡터의 위치를 랜덤으로 초기화 <br>
(3) 입력층과 가장 가까운 노드를 찾는다(Winning node). <br>
(4) Winning node 결정 이후, 반복된 학습을 통해 연결강도(W)가 결정.
- Wnew = Wold + a(X-Wold) <br>

(5) 구성
- vector of nodes for input
- array of nodes as output map
- a matrix of connections <br>

![대체 텍스트](https://i.imgur.com//eHUVAtr.png)


* Reference <br>
1. https://ratsgo.github.io/machine%20learning/2017/05/01/SOM/ <br>
2. http://www.incodom.kr/Self-organizing_map%28SOR%29 <br>


###3. LVQ(Learning Vector Quantization)

(1) 입력 벡터를 가장 유사한 참조 벡터로 군집화하는 인공신경망 <br>
(2) 참조 벡터(reference vector): 경쟁학습을 통해 형성되는 입력 벡터의 그룹 <br>
(3) 유클리드 거리 기반의 경쟁학습을 통해 클러스터의 중심을 수정하며 학습 진행

### - LVQ 동작 방법

(1) 클러스터(참조 벡터)의 수, 학습률 설정 <br>
(2) 클러스터의 중심을 0 ~ 1의 임의의 값으로 초기화. <br>
(3) k를 0으로 초기화 <br>
(4) x_k를 인공신경망의 입력으로 설정. <br>
(5) 경쟁학습을 통해 클러스터를 선택 <br>
(6) 선택된 클러스터의 중심을 수정 <br>
(7-1) 학습한 결과가 종료 조건을 만족 -> 알고리즘 종료 <br>
(7-2) 종료 조건 만족X -> k=k+1, (4)부터 다시 수행

### - SOM과 LVQ의 차이점

- SOM: preserve the data topology by mapping similar data items to same cell on the grid. <br> 유사 데이터를 같은 셀에 맵핑하여 유사성 유지. <br><br>
- LVQ: don't take into account data topology. -> use pre-assigned cluster labels to data items. <br> 데이터 유사성을 고려하지 않으므로 미리 데이터 아이템에 클러스터 할당

In [0]:

import numpy as np

# train_lvq: trains an lvq system using the given training data and
# corresponding labels. Run the desired number of epochs using the
# given learning rate. Optional validation set to monitor performance.
def train_lvq(data, labels, num_epochs, learning_rate, validation_data=None, validation_labels=None):
    # Get unique class labels.
    num_dims = data.shape[1]
    labels = labels.astype(int)
    unique_labels = list(set(labels))

    num_protos = len(unique_labels)
    prototypes = np.empty((num_protos, num_dims))
    proto_labels = []

    # Initialize prototypes using class means.
    for i in unique_labels:
        class_data = data[labels == i, :]

        # Compute class mean.
        mean = np.mean(class_data, axis=0)

        prototypes[i] = mean
        proto_labels.append(i)

    # Loop through data set.
    for epoch in range(0, num_epochs):
        for fvec, lbl in zip(data, labels):
            # Compute distance from each prototype to this point
            distances = list(np.sum(np.subtract(fvec, p)**2) for p in prototypes)
            min_dist_index = distances.index(min(distances))

            # Determine winner prototype.
            winner = prototypes[min_dist_index]
            winner_label = proto_labels[min_dist_index]

            # Push or repel the prototype based on the label.
            if winner_label == lbl:
                sign = 1
            else:
                sign = -1

            # Update winner prototype
            prototypes[min_dist_index] = np.add(prototypes[min_dist_index], np.subtract(fvec, winner) * learning_rate * sign)

        # Use validation set to test performance.
        val_err = 0
        if validation_labels is not None:
            for fvec, lbl in zip(validation_data, validation_labels):
                distances = list(np.sum(np.subtract(fvec, p) ** 2) for p in prototypes)
                min_dist_index = distances.index(min(distances))

                # Determine winner prototype label
                winner_label = proto_labels[min_dist_index]

                # Check if labels match
                if not winner_label == lbl:
                    val_err = val_err + 1

            val_err = val_err / len(validation_labels)
            print("Epoch " + str(epoch) + ". Validation error: " + str(val_err))
        else:
            print("Epoch " + str(epoch))


    return (prototypes, proto_labels)

In [23]:
!pip install neurolab

     |████████████████████████████████| 655kB 1.4MB/s 
  Created wheel for neurolab: filename=neurolab-0.3.5-cp36-none-any.whl size=22180 sha256=60a6cc382265f5124c41eacd81811b1e1795a7771556ed39d8da20ca147df0b2
  Stored in directory: /root/.cache/pip/wheels/c6/8f/37/32ab1cf4d601dc0bc49d7241012a4292db4b343bebff5b68e6
Successfully built neurolab


In [26]:
"""
Example of use LVQ network
==========================

"""
import numpy as np
import neurolab as nl

# Create train samples
input = np.array([[-3, 0], [-2, 1], [-2, -1], [0, 2], [0, 1], [0, -1], [0, -2], 
                                                        [2, 1], [2, -1], [3, 0]])
target = np.array([[1, 0], [1, 0], [1, 0], [0, 1], [0, 1], [0, 1], [0, 1], 
                                                        [1, 0], [1, 0], [1, 0]])

# Create network with 2 layers: 4 neurons in input layer(Competitive)
# and 2 neurons in output layer(liner)
net = nl.net.newlvq(nl.tool.minmax(input), 4, [0.6, 0.4])
# Train network
error = net.train(input, target, epochs=1000, goal=-1)

# Plot result
import pylab as pl
xx, yy = np.meshgrid(np.arange(-3, 3.4, 0.2), np.arange(-3, 3.4, 0.2))
xx.shape = xx.size, 1
yy.shape = yy.size, 1
i = np.concatenate((xx, yy), axis=1)
o = net.sim(i)
grid1 = i[o[:, 0]>0]
grid2 = i[o[:, 1]>0]

class1 = input[target[:, 0]>0]
class2 = input[target[:, 1]>0]

pl.plot(class1[:,0], class1[:,1], 'bo', class2[:,0], class2[:,1], 'go')
pl.plot(grid1[:,0], grid1[:,1], 'b.', grid2[:,0], grid2[:,1], 'gx')
pl.axis([-3.2, 3.2, -3, 3])
pl.xlabel('Input[:, 0]')
pl.ylabel('Input[:, 1]')
pl.legend(['class 1', 'class 2', 'detected class 1', 'detected class 2'])
pl.show()

TypeError: ignored

In [28]:
!pip install neupy

     |████████████████████████████████| 235kB 1.4MB/s 
     |████████████████████████████████| 92.6MB 60kB/s 
     |████████████████████████████████| 368kB 39.9MB/s 
     |████████████████████████████████| 3.2MB 34.8MB/s 
  Found existing installation: tensorflow-estimator 2.2.0rc0
    Uninstalling tensorflow-estimator-2.2.0rc0:
      Successfully uninstalled tensorflow-estimator-2.2.0rc0
  Found existing installation: tensorboard 2.2.0
    Uninstalling tensorboard-2.2.0:
      Successfully uninstalled tensorboard-2.2.0
  Found existing installation: tensorflow 2.2.0rc2
    Uninstalling tensorflow-2.2.0rc2:
      Successfully uninstalled tensorflow-2.2.0rc2
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
  Found existing installation: progressbar2 3.38.0
    Uninstalling progressbar2-3.38.0:
      Successfully uninstalled progressbar2-3.38.0


In [30]:
import numpy as np
from neupy import algorithms
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1], [2, 2], [1, 2]])
y = np.array([0, 0, 0, 1, 1, 1])
lvqnet = algorithms.LVQ(n_inputs=2, n_classes=2)
lvqnet.train(X, y, epochs=100)
lvqnet.predict([[2, 1], [-1, -1]])

array([1, 0])

* Reference <br>
1. https://untitledtblog.tistory.com/50?category=667127 <br> 
2. http://facweb.cs.depaul.edu/mobasher/classes/csc426/review/categorization.pdf <br>


###4. 심층학습(DNN, Deep Neural Network)